# Visualizing Experimental Results

This notebook is used to plot experimental results as shown in Vision-based Landing Guidance through Tracking and Orientation Estimation.

In [ ]:
from matplotlib.ticker import FuncFormatter
from matplotlib.patches import Patch

In [ ]:
class AsinhScale:
    def __init__(self, a=1):
        self.a = a

    def transform(self, x):
        return np.arcsinh(x / self.a)

    def inverse(self, y):
        return np.sinh(y) * self.a

    def format(self, x, pos):
        return f'{int(x):d}'

def box_plot(sd, sd_hat_gt, sd_hat_kp, ypr, ypr_hat_gt, ypr_hat_kp, airport, runway, test_scenery, scale="linear"):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5), sharex=False)
    err_gt = np.degrees(copy.deepcopy(ypr-ypr_hat_gt))
    err_kp = np.degrees(copy.deepcopy(ypr-ypr_hat_kp))
    sd_err_gt = sd - sd_hat_gt
    sd_err_kp = sd - sd_hat_kp

    print(f"GT Yaw error mean: {np.mean(err_gt[:, 0])} +- {np.std(err_gt[:, 0])}")
    print(f"GT Pitch error mean: {np.mean(err_gt[:, 1])} +- {np.std(err_gt[:, 1])}")
    print(f"GT Roll error mean: {np.mean(err_gt[:, 2])} +- {np.std(err_gt[:, 2])}")
    print(f"GT Slant Distance error mean: {np.mean(abs(sd_err_gt[:]))} +- {np.std(abs(sd_err_gt[:]))}\n")

    print(f"DKP Yaw error mean: {np.mean(err_kp[:, 0])} +- {np.std(err_kp[:, 0])}")
    print(f"DKP Pitch error mean: {np.mean(err_kp[:, 1])} +- {np.std(err_kp[:, 1])}")
    print(f"DKP Roll error mean: {np.mean(err_kp[:, 2])} +- {np.std(err_kp[:, 2])}")
    print(f"DKP Slant Distance error mean: {np.mean(abs(sd_err_kp[:]))} +- {np.std(abs(sd_err_kp[:]))}\n")

    positions1 = [1, 2]   # First set of boxes
    positions2 = [5, 6]  # Second set of boxes, slightly shifted
    positions3 = [9, 10]  # Second set of boxes, slightly shifted

    widths = 0.8

    ax1.boxplot(err_kp[mask, 0], positions=[positions1[0]], widths=widths, patch_artist=True, medianprops=dict(color='lightgreen'))
    ax1.boxplot(err_kp[mask, 1], positions=[positions2[0]], widths=widths, patch_artist=True, medianprops=dict(color='lightgreen'))
    ax1.boxplot(err_kp[mask, 2], positions=[positions3[0]], widths=widths, patch_artist=True, medianprops=dict(color='lightgreen'))
    ax2.plot(sd_hat_kp[mask], label='detected-keypoint-based estimation')
    
    ax1.boxplot(err_gt[mask, 0], positions=[positions1[1]], widths=widths, patch_artist=True,boxprops=dict(facecolor='orange'), medianprops=dict(color='lightgreen'))
    ax1.boxplot(err_gt[mask, 1], positions=[positions2[1]], widths=widths, patch_artist=True,boxprops=dict(facecolor='orange'), medianprops=dict(color='lightgreen'))
    ax1.boxplot(err_gt[mask, 2], positions=[positions3[1]], widths=widths, patch_artist=True,boxprops=dict(facecolor='orange'), medianprops=dict(color='lightgreen'))
    ax2.plot(sd[mask], label='ground truth')

    ax1.set_xticks([1.5, 5.5, 9.5])
    ax1.set_xticklabels(['Yaw', 'Pitch', 'Roll'], fontsize=16)

    legend_elements = [Patch(edgecolor='black', label='detected-keypoint-based estimation'),
                   Patch(facecolor='orange', edgecolor='black', label='ground-truth-based estimation')]
    ax1.legend(handles=legend_elements, fontsize=13, loc='best')

    ax1.set_ylabel("Error (°)", fontsize=16)
    ax1.tick_params(axis='y', labelsize=14)
    ax2.tick_params(axis='both', labelsize=14)

    if scale == "Asinh":
        asinh = AsinhScale(a=0.1)
        ax1.set_yscale('function', functions=(asinh.transform, asinh.inverse))
        ax1.yaxis.set_major_formatter(FuncFormatter(asinh.format))
        custom_ticks = [-64, -8, -1, 0, 1, 8, 64]
        ax1.set_yticks(custom_ticks)

    ax2.set_xlabel('Frame', fontsize=16)
    ax2.set_ylabel('Slant Distance to Runway (NM)', fontsize=16)

    # Set titles for each subplot
    ax1.set_title('Orientation Estimation Error', fontsize=24, weight='bold')
    ax2.set_title('Slant Distance Estimation', fontsize=24, weight='bold')
    ax2.legend(loc='best',fontsize=13)

    # Adjust layout for better appearance
    fig.subplots_adjust(top=0.8, wspace=0.3)
    ax1.yaxis.grid(True)

    plt.savefig('box_plot_'+airport+runway+test_scenery+'_no_title.pdf', bbox_inches='tight', pad_inches=0)

    # Show the plot
    plt.show()

In [ ]:
box_plot(slant_distance_gt, estimated_distance_gt, estimated_distance_kp, pose_gt, estimated_pose_gt, estimated_pose_kp, airport_name, runway, scenario)